# Exercise Gridworld

## Table of Contents
* [Introduction](#Introduction) 
* [Requirements](#Requirements) 
  * [Knowledge](#Knowledge) 
  * [Prerequisites](#Prerequisites) 
* [Python Modules](#Python-Modules)
* [Exercises](#Exercises) 
* [Literature](#Literature) 
* [Licenses](#Licenses) 

## Introduction

In this notebook you will train an agent to find a finish grid in a 4x4 Gridworld. This fairly easy examble will demonstrate the basic princibles of reinforcement learning in an agent-enviroment interaction system based on the Bellman equation. Along the exercises you will define all parts of the Bellman equation and in the last step solve this equation and therefore train an agent.  

## Requirements

### Knowledge

To solve this notebook you should aquire knowledge about fMDP (finite Markov Decision Processe), agent-enviroment interaction, Bellman equation and the policy iteration algorithm to solve the Bellman equation. 

### Prerequisites

Read `SUT18`  chapter 3 and 4 to gain knowledge about the mentioned topics and terms. `SUT18` is the standard literature for reinforcment learning and the basis for this and following notebooks.

## Python Modules

In [258]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from hashlib import sha1

## Gridworld

The gridworld we consider here is a simple 4x4 grid with a possible start state $S$ and a fixed endstate $E$ in the bottom right corner.

In [259]:
print("Gridworld")
print()
print(" -- -- -- --")
print("|S |  |  |  |")
print(" -- -- -- --")
print("|  |  |  |  |")
print(" -- -- -- --")
print("|  |  |  |  |")
print(" -- -- -- --")
print("|  |  |  |E |")
print(" -- -- -- --")

Gridworld

 -- -- -- --
|S |  |  |  |
 -- -- -- --
|  |  |  |  |
 -- -- -- --
|  |  |  |  |
 -- -- -- --
|  |  |  |E |
 -- -- -- --


### States

A 4x4 grid gives 16 states in total, which we can numerate from 1 to 16. As we will see later these will be the matrix rows, respectively columns when we define our enviroment behavior.

In [260]:
print(" -- -- -- --")
print("|1 |2 |3 |4 |")
print(" -- -- -- --")
print("|5 |6 |7 |8 |")
print(" -- -- -- --")
print("|9 |10|11|12|")
print(" -- -- -- --")
print("|13|14|15|16|")
print(" -- -- -- --")

 -- -- -- --
|1 |2 |3 |4 |
 -- -- -- --
|5 |6 |7 |8 |
 -- -- -- --
|9 |10|11|12|
 -- -- -- --
|13|14|15|16|
 -- -- -- --


### Action

The actions our agent can take are **up, down, right and left**. Later this action will be encoded with numpy array indices **0,1,2 and 3**. Moving against a wall will leave the agent on his state.

# Markov decision process

To get a first inside into our envirement, in the next few exercises you will build transition matrices which contain the behavior of our model/world the agent is moving in.
For that consider a random policy , meaning the agent will take any action with a probability of 25%. As said before hitting a wall will leave the agent in the actual state.


### Exercise
#### Task 1 

Consider each state as a vector with start state given by (1,...,0), so we are with probability 1 in state 1. Find the transition matrix given a random policy for that given 4x4 gridworld. 

Here the endstate **doesn't** matter, so the agent there will behave the same as in any other state, we are not learning anything yet. 

**Hint:**

Each row represents the current state and each column a possible furture state.
Your matrix should have shape (16,16).

In [261]:
T = np.array([
    [0.5, 0.25, 0., 0., 0.25, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0.25, 0.25, 0.25, 0., 0., 0.25, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0.25, 0.25, 0.25, 0., 0., 0.25, 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0.25, 0.5, 0., 0., 0., 0.25, 0., 0., 0., 0., 0., 0., 0., 0.],
    [0.25, 0., 0., 0., 0.25, 0.25, 0., 0., 0.25, 0., 0., 0., 0., 0., 0., 0.],
    [0., 0.25, 0., 0., 0.25, 0., 0.25, 0., 0., 0.25, 0., 0., 0., 0., 0., 0.],
    [0., 0., 0.25, 0., 0., 0.25, 0., 0.25, 0., 0., 0.25, 0., 0., 0., 0., 0.],
    [0., 0., 0., 0.25, 0., 0., 0.25, 0.25, 0., 0., 0., 0.25, 0., 0., 0., 0.],
    [0., 0., 0., 0., 0.25, 0., 0., 0., 0.25, 0.25, 0., 0., 0.25, 0., 0., 0.],
    [0., 0., 0., 0., 0., 0.25, 0., 0., 0.25, 0., 0.25, 0., 0., 0.25, 0., 0.],
    [0., 0., 0., 0., 0., 0., 0.25, 0., 0., 0.25, 0., 0.25, 0., 0., 0.25, 0.],
    [0., 0., 0., 0., 0., 0., 0., 0.25, 0., 0., 0.25, 0.25, 0., 0., 0., 0.25],
    [0., 0., 0., 0., 0., 0., 0., 0., 0.25, 0., 0., 0., 0.5, 0.25, 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.25, 0., 0., 0.25, 0.25, 0.25, 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.25, 0., 0., 0.25, 0.25, 0.25],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.25, 0., 0., 0.25, 0.5]
])

Test your matrix here.

In [262]:
hash_T = sha1(T).hexdigest()
assert str(hash_T) == "42b00d22d434f693a40dc196f0dee056383c6672"

#### Task 2

Taken two random steps , what is the probability to go to state 3 and what is the probability to go to state 6 ?

In [263]:
print('The probability to go to state 3 and state 6 are both 1/16')

The probability to go to state 3 and state 6 are both 1/16


#### Task 3

If we do many steps, lets say 100+, what is the probability for each possible state?

In [264]:
print('I am not sure, but I guess they are all 1/16')
print('At first, the probabilties are unstable. But after many steps, they converge to 1/16')

I am not sure, but I guess they are all 1/16
At first, the probabilties are unstable. But after many steps, they converge to 1/16


#### Task 4

Now lets say the agent is a bit more clever and let him "know" that the end is somewhere down to the right. So he only goes right by 50% and down by 50%.

* Find the transition matrix.
* At how many steps we can be sure we reached the end?
* What would be our optimal solution?

In [265]:
T_clever = np.array([
    [0., 0.5, 0., 0., 0.5, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0.5, 0., 0., 0.5, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0.5, 0., 0., 0.5, 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0.5, 0., 0., 0., 0.5, 0., 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0.5, 0., 0., 0.5, 0., 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0.5, 0., 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0.5, 0., 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0., 0.5, 0., 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0.5, 0., 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0.5, 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0.5, 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0., 0., 0., 0.5],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0.5, 0., 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0.5, 0.],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.5, 0.5],
    [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1]
])

In [266]:
print('With 6 steps we can get the end')
print("There are several optimal solutions. All of them keep going without 'hitting the wall'")

With 6 steps we can get the end
There are several optimal solutions. All of them keep going without 'hitting the wall'


Test your matrix here.

In [267]:
hash_T_clever = sha1(T_clever).hexdigest()
assert str(hash_T_clever) == "4597d1bfacf69ab6c021bc4f14e25469d96de872"

# Solution with reinforcement learning

To solve this gridworld problem with reinforcement learning we use the Bellman equation and policy iteration found on page 80 in `SUT18`.

The Bellman equation is given by
$$
v_{\pi}(s) = \sum_a \pi(a|s) \sum_{s'} \sum_r p(s',r|s,a)[r + \gamma v_\pi(s')].
$$

## Model 

For $p(s',r|s,a)$ (model) we have a 16x16x4 matrix , so a 16x16 transition matrix for each action (**up,down,right,left**) which we number (**0,1,2,3**). From the previous exercises we know how to set up these matrixes.

**Hint:** For each action taken in a state the agent transitions into another state corresbonding to the action taken. So each row should have exactly one $1$ entry.

#### Task 5

Find for each action the corresbonding transition matrix.

In [268]:
#p(s', r|s, a=up)
P_up = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
])

P_down = np.array([
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])

P_right = np.array([
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
])


P_left = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],   
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
])

P_model = np.array([P_up,P_down,P_left,P_right]) 
print(P_model.shape)

(4, 16, 16)


#### Solution

Test your matrix here.

In [269]:
hash_P_model = sha1(P_model).hexdigest()
assert str(hash_P_model) == "afd11e846b0d2622ab11dc1a5505077623ac9d5f"

## Reward
In order for the agent to learn we have to define a good reward strategy.
A simple choice is  -1 for each transition to any state, which is not the end state, and +10 for end state.

In [270]:
reward = -1*np.ones(16)
reward[-1] = 10
print(reward)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. 10.]


## $\gamma$  -value

The $\gamma$  -value defines how important future rewards are. A default choice for problems like gridworld is 0.9, so future rewards are important for the agent.

In [271]:
gamma = 0.9

# Training the agent

## Policy evaluation

In [272]:
#Plots the values for all 16 states in our gridworld
def show_values(V):
    #print("Values")
    print()
    print("-----------------------------")
    print("|",round(V[0],1),"|",round(V[1],1),"|",round(V[2],1),"|",round(V[3],1),"|")
    print("-----------------------------")
    print("|",round(V[4],1),"|",round(V[5],1),"|",round(V[6],1),"|",round(V[7],1),"|")
    print("-----------------------------")
    print("|",round(V[8],1),"|",round(V[9],1),"|",round(V[10],1),"|",round(V[11],1),"|")
    print("-----------------------------")
    print("|",round(V[12],1),"|",round(V[13],1),"|",round(V[14],1),"|",round(V[15],1),"|")
    print("-----------------------------")
    print()

#### Task 6

Study the pseudo-algorithm for **Iterative Policy Evaluation** on page 75 in `SUT18` and implement\complete the `policy_evaluation` function below for our problem.

In [273]:
def policy_evaluation(V_k,policy,P_model,reward,gamma):
    
    """
    Args:
    
    index abbreviations:
    
    - state s
    - action a
    - next state s_nxt
    
    
        V_k: current values, size 16 array V_k[s]
        
        policy: current policy, 16x4 ndarray policy[s,a]
        
        P_model: fixed model behaviour, 4x16x16 ndarray P_model[a,s,s_nxt]
        
        reward: fixed reward model, size 16 array reward[s_nxt]
        
        gamma: fixed discount rate , float
    
    Return:
    
        V_k: updated values (overwritten old values)
    """
    #r is fixed, so only 1 layer loop instead of 2-layers loop
    delta = 1e6
    a = np.argmax(policy, axis=1)
    while delta > 1e-6:
        v = V_k
        for s in range(len(V_k)):
            V_k[s] = sum([P_model[a[s], s, s_nxt]*(reward[s_nxt]+gamma*v[s_nxt]) for s_nxt in range(len(V_k))])
        delta = np.max(abs(V_k - v))
        delta = max(delta, np.max(abs(V_k - v)))
    return V_k

To test your implementation, run the following code.

In [274]:
# print("If nothing happend you are good!")
# test_V_k = np.arange(16)
# test_policy = 0.25*np.ones((16,4))
# update_V_K = policy_evaluation(test_V_k,test_policy,P_model,reward,gamma)
# hash_V_K = sha1(update_V_K).hexdigest()
# assert str(hash_V_K) == "ee70b34908511bc6e2b44f2fac8151d7933cf6e7"

## Policy improvement

In [275]:
#Plots the "best" action given by the actual policy for all 16 states in our gridworld
def show_policy(p):
    #print("Policy")
    print()
    print("Actions with highest probability")
    print("_______")
    print("a := all possible actions")
    
    
    def symbol(a): 
        if (a[0]==a[1]) and (a[1]==a[2]) and (a[2]==a[3]):
            return "a"
        if np.argmax(a)==0:
            return "↑"
        if np.argmax(a)==1:
            return "↓"
        if np.argmax(a)==2:
            return "←"
        if np.argmax(a)==3:
            return "→"
        
    print("-----------------")
    print("| "+symbol(p[0])+" | "+symbol(p[1])+" | "+symbol(p[2])+" | "+symbol(p[3])+" |")
    print("-----------------")
    print("| "+symbol(p[4])+" | "+symbol(p[5])+" | "+symbol(p[6])+" | "+symbol(p[7])+" |")
    print("-----------------")
    print("| "+symbol(p[8])+" | "+symbol(p[9])+" | "+symbol(p[10])+" | "+symbol(p[11])+" |")
    print("-----------------")
    print("| "+symbol(p[12])+" | "+symbol(p[13])+" | "+symbol(p[14])+" |>E<|")
    print("-----------------")

#### Task 7

Study the pseudo-algorithm for **Policy Improvement** on page 80 in `SUT18` and implement\complete the `policy_improvement` function below for our problem.

In [276]:
def policy_improvement(V_k,policy,P_model,reward,gamma):
    """
    Args:
    
    index abbreviations:
    
    - state s
    - action a
    - next state s_nxt
    
    
        V_k: current values, size 16 array V_k[s]
        
        policy: current policy, 16x4 ndarray policy[s,a]
        
        P_model: fixed model behaviour, 16x16x4 ndarray P_model[a,s,s_nxt]
        
        reward: fixed reward model, size 16 array reward[s_nxt]
        
        gamma: fixed discount rate , float
    
    Return:
    
        policy: updated policy (overwritten old policy)
    """
    
    for s in range(policy.shape[0]):
        for a in range(policy.shape[1]):
            policy[s,a] = sum([P_model[a, s, s_nxt]*(reward[s_nxt]+gamma*V_k[s_nxt]) for s_nxt in range(len(V_k))])
            
    return policy
        
    

To test your implementation, run the following code.

In [277]:
# print("If nothing happend you are good!")
# test_V_k = np.arange(16)
# test_policy = 0.25*np.ones((16,4))
# update_policy = policy_improvement(test_V_k,test_policy,P_model,reward,gamma)
# hash_policy = sha1(update_policy).hexdigest()
# assert str(hash_policy) == "5e0dfa1ae264b0d7d829bb96d55bd9bf87b932d3"

### Initial Values and random Policy for training. 

Here we initialize and show the starting values and policy our agent uses to learn. 

In [278]:
#V_k = np.zeros(16,dtype=float)
policy = 0.25*np.ones((16,4))

V_k = np.random.rand(16)
show_values(V_k)

# generate complete random policy
for s in range(16):
    act = np.random.rand(4)
    policy[s] = act/sum(act)
    
show_policy(policy)
print()
print(policy)


-----------------------------
| 0.9 | 0.5 | 0.6 | 0.5 |
-----------------------------
| 0.9 | 0.5 | 0.2 | 1.0 |
-----------------------------
| 0.1 | 0.1 | 0.1 | 0.5 |
-----------------------------
| 0.2 | 0.1 | 0.0 | 0.1 |
-----------------------------


Actions with highest probability
_______
a := all possible actions
-----------------
| → | → | → | ← |
-----------------
| → | ↓ | ↑ | ← |
-----------------
| ← | ↑ | → | ↓ |
-----------------
| ↑ | ↑ | ← |>E<|
-----------------

[[0.26932033 0.32498877 0.02305311 0.38263779]
 [0.25123393 0.37039628 0.00738458 0.37098521]
 [0.24853615 0.22741501 0.23012126 0.29392758]
 [0.21172657 0.06266553 0.55353906 0.17206884]
 [0.34131982 0.08448365 0.21950542 0.35469111]
 [0.12681998 0.61855507 0.1415013  0.11312365]
 [0.33567901 0.17599959 0.19259559 0.29572581]
 [0.05851828 0.26538316 0.36183154 0.31426702]
 [0.29545355 0.34092005 0.35528801 0.00833839]
 [0.30169993 0.28932468 0.16173275 0.24724264]
 [0.23125067 0.31261911 0.10219704 0.353933

## Policy iteration loop

Run the following code to train the agend. This loop is the full **policy iteration algorithm** given on page 80 in `SUT18`. If your implementations above work you should the see learning.

In [279]:
policy_stable = False
max_loops = 10
ite = 0 

while not policy_stable:
    
    print("------------ Iteration "+str(ite)+"---------------")
    print()
    
    old_policy = copy.copy(policy)
    
    V_k = policy_evaluation(V_k,policy,P_model,reward,gamma)
    policy = policy_improvement(V_k,policy,P_model,reward,gamma)
    
    show_values(V_k)
    show_policy(policy)
    
    # check if stable
    policy_stable = np.all(policy==old_policy)
        
    ite += 1
    if ite > max_loops:
        break
    print()

print("--> Training Done!")
print()
print("Final policy")
print()
print(policy)

    

------------ Iteration 0---------------


-----------------------------
| -0.5 | -0.5 | -0.6 | -1.5 |
-----------------------------
| -0.6 | -0.9 | -1.5 | -2.4 |
-----------------------------
| -0.9 | -1.8 | -0.6 | 10.1 |
-----------------------------
| -1.8 | -2.6 | -3.4 | -4.0 |
-----------------------------


Actions with highest probability
_______
a := all possible actions
-----------------
| → | ↑ | ← | ← |
-----------------
| ↑ | ↑ | ↓ | ↓ |
-----------------
| ↑ | → | → | → |
-----------------
| ↑ | ↑ | → |>E<|
-----------------

------------ Iteration 1---------------


-----------------------------
| -1.4 | -1.4 | -2.3 | -3.1 |
-----------------------------
| -2.3 | -2.3 | -1.5 | 8.1 |
-----------------------------
| -3.1 | -1.5 | 8.1 | 8.1 |
-----------------------------
| -3.8 | -2.3 | 6.4 | 6.3 |
-----------------------------


Actions with highest probability
_______
a := all possible actions
-----------------
| ↑ | ↑ | ← | ↓ |
-----------------
| ↑ | ↑ | ↓ | ↓ |
--------

## Literature


<table>
    <tr>
        <td>
            <a name="SUT18"></a>[SUT18]
        </td>
        <td>
            Richard S. Sutton and Andrew G. Barto, “Reinforcement Learning: An Introduction
” ,2nd edition,2018. 
        </td>
    </tr>
   
</table>

## Licenses

### Notebook License (CC-BY-SA 4.0)

*The following license applies to the complete notebook, including code cells. It does however not apply to any referenced external media (e.g., images).*

Exercise Gridworld

Oliver Fischer

is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).<br/>
Based on a work at https://gitlab.com/deep.TEACHING.


### Code License (MIT)

*The following license only applies to code cells of the notebook.*

Copyright 2019  Oliver Fischer

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.